## Menentukan parameter sebelum modulasi OFDM

In [14]:
import numpy as np
import scipy.interpolate
import scipy

In [15]:
K = 64 # number of OFDM subcarriers
CP = K//4 #print(CP);  # length of the cyclic prefix: 25% of the block
P = CP//2 # number of pilot carriers per OFDM block

## Menentukan Value setiap komponen: Carriers, Pilot

In [16]:
pilotValue = 3+3j # The known value each pilot transmits

allCarriers = np.arange(K)  # indices of all subcarriers ([0, 1, ... K-1])

pilotCarriers = allCarriers[::K//P] # Pilots is every (K/P)th carrier.

# For convenience of channel estimation, let's make the last carriers also be a pilot
pilotCarriers = np.hstack([pilotCarriers, np.array([allCarriers[-1]])])
P = P+1

# data carriers are all remaining carriers
dataCarriers = np.delete(allCarriers, pilotCarriers)

In [17]:
print("pilot value:", pilotValue)
print("all carriers:", allCarriers)
print("pilot carriers:", pilotCarriers)
print("data carrires:", dataCarriers)

pilot value: (3+3j)
all carriers: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
pilot carriers: [ 0  8 16 24 32 40 48 56 63]
data carrires: [ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 17 18 19 20 21 22 23 25 26 27
 28 29 30 31 33 34 35 36 37 38 39 41 42 43 44 45 46 47 49 50 51 52 53 54
 55 57 58 59 60 61 62]


## Menentukan μ (OFDM Symbol) dan Payloadbits per OFDM

In [18]:
mu = 4 # untuk 16QAM
payloadBits_per_OFDM = len(dataCarriers) * mu

## Membuat Mapping Table

In [19]:
mapping_table = {
    (0,0,0,0) : -3-3j,
    (0,0,0,1) : -3-1j,
    (0,0,1,0) : -3+3j,
    (0,0,1,1) : -3+1j,
    (0,1,0,0) : -1-3j,
    (0,1,0,1) : -1-1j,
    (0,1,1,0) : -1+3j,
    (0,1,1,1) : -1+1j,
    (1,0,0,0) :  3-3j,
    (1,0,0,1) :  3-1j,
    (1,0,1,0) :  3+3j,
    (1,0,1,1) :  3+1j,
    (1,1,0,0) :  1-3j,
    (1,1,0,1) :  1-1j,
    (1,1,1,0) :  1+3j,
    (1,1,1,1) :  1+1j
}

## Menentukan Channel Response, Signal to Noise Radio in desible (SNRdb) pada receiver

In [20]:
channelResponse = np.array([1, 0, 0.3+0.3j])
H_exact = np.fft.fft(channelResponse, K)
SNRdb = 24

In [24]:
bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM, )) # input data dalam bentuk bits

In [25]:
def SP(bits):
    return bits.reshape((len(dataCarriers), mu))

bits_SP = SP(bits)

In [28]:
print(bits_SP)

[[0 1 1 0]
 [0 0 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 0 1 0]
 [1 0 1 1]
 [0 0 1 1]
 [1 0 0 0]
 [1 1 0 0]
 [0 0 1 0]
 [1 1 1 0]
 [1 1 0 0]
 [1 0 1 1]
 [0 1 0 1]
 [1 0 1 0]
 [0 1 1 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [1 1 0 1]
 [0 1 0 1]
 [1 0 1 1]
 [1 0 0 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 0 1 1]
 [1 1 0 1]
 [0 0 0 0]
 [1 1 1 1]
 [1 0 0 0]
 [0 1 1 1]
 [1 0 0 0]
 [0 0 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [0 1 0 1]
 [1 0 1 0]
 [1 0 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 1]
 [1 1 0 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 1 0]
 [0 0 0 1]
 [1 0 1 1]
 [1 0 0 1]
 [0 0 1 0]
 [1 1 1 1]
 [1 0 1 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 1 0]]


In [27]:
print(len(bits_SP))

55


In [26]:
def Mapping(bits):
    return np.array([mapping_table[tuple(b)] for b in bits])

QAM = Mapping(bits_SP)

In [29]:
print(QAM)
print(len(QAM))

[-1.+3.j -3.+3.j  3.+1.j  1.-3.j  3.+3.j  3.+1.j -3.+1.j  3.-3.j  1.-3.j
 -3.+3.j  1.+3.j  1.-3.j  3.+1.j -1.-1.j  3.+3.j -1.+3.j -1.-3.j -1.-1.j
 -1.+3.j  1.-1.j -1.-1.j  3.+1.j  3.-1.j  1.+3.j  1.+1.j  3.+1.j  1.-1.j
 -3.-3.j  1.+1.j  3.-3.j -1.+1.j  3.-3.j -3.-3.j  1.+1.j  1.+1.j -1.-1.j
  3.+3.j  3.-3.j -3.-3.j -1.-3.j -3.-1.j  1.-3.j -3.-1.j -1.-1.j -3.-1.j
 -1.+3.j -3.-1.j  3.+1.j  3.-1.j -3.+3.j  1.+1.j  3.+1.j -3.-3.j -3.+3.j
 -3.+3.j]
55


In [30]:
def OFDM_symbol(QAM_payload):
    symbol = np.zeros(K, dtype=complex) # the overall K subcarriers
    symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers 
    symbol[dataCarriers] = QAM_payload  # allocate the pilot subcarriers
    return symbol

OFDM_data = OFDM_symbol(QAM)

In [31]:
print(len(OFDM_data))
print(OFDM_data)

64
[ 3.+3.j -1.+3.j -3.+3.j  3.+1.j  1.-3.j  3.+3.j  3.+1.j -3.+1.j  3.+3.j
  3.-3.j  1.-3.j -3.+3.j  1.+3.j  1.-3.j  3.+1.j -1.-1.j  3.+3.j  3.+3.j
 -1.+3.j -1.-3.j -1.-1.j -1.+3.j  1.-1.j -1.-1.j  3.+3.j  3.+1.j  3.-1.j
  1.+3.j  1.+1.j  3.+1.j  1.-1.j -3.-3.j  3.+3.j  1.+1.j  3.-3.j -1.+1.j
  3.-3.j -3.-3.j  1.+1.j  1.+1.j  3.+3.j -1.-1.j  3.+3.j  3.-3.j -3.-3.j
 -1.-3.j -3.-1.j  1.-3.j  3.+3.j -3.-1.j -1.-1.j -3.-1.j -1.+3.j -3.-1.j
  3.+1.j  3.-1.j  3.+3.j -3.+3.j  1.+1.j  3.+1.j -3.-3.j -3.+3.j -3.+3.j
  3.+3.j]


In [32]:
def IFFT(OFDM_data):
  return np.fft.ifft(OFDM_data)

OFDM_time = IFFT(OFDM_data)

In [34]:
print(len(OFDM_time))
print(OFDM_time)

64
[ 0.53125   +0.4375j     -0.39825862+0.66254641j  0.43680491+0.3668746j
  0.01595144+0.18609162j -0.2835711 -0.071391j   -0.24169586+0.60786338j
  0.02588168+0.21793871j  0.02111343-0.15019061j  0.34911165+0.55713835j
  0.32420587-0.25392738j -0.22744271+0.10062154j  0.28612407+0.33458445j
  0.02822113+0.071391j    0.28937808-0.72679748j  0.15741834-0.5151882j
  0.66245054-0.31682454j  0.03125   +0.125j      -0.49864865-0.35884965j
  0.01231268+0.12659221j -0.49654335-0.59522007j  0.10109054-0.05373907j
  0.60669567-0.17715569j  0.05676237+0.17994809j  0.38409088+0.09987563j
  0.1982233 +0.55713835j -0.12656122-0.00910687j -0.06849716+0.18115378j
  0.37812416+0.23806889j -0.22021674-0.1970449j  -0.47640117+0.04417121j
  0.05591575-0.52105741j -0.3989296 -0.08306062j  0.53125   +0.3125j
 -0.05612707-0.10500062j -0.18154691-0.3835098j  -0.42751135-0.20018177j
 -0.14006481+0.30308517j -0.06948787-0.02625807j  0.11352098+0.19335715j
 -0.18395625-0.00216089j  0.52588835+0.38036165j  0.21

In [35]:
def addCP(OFDM_time):
  cp = OFDM_time[-CP:]               # take the last CP samples ...
  return np.hstack([cp, OFDM_time])  # ... and add them to the beginning

OFDM_withCP = addCP(OFDM_time)

In [36]:
print(len(OFDM_withCP))
print(OFDM_withCP)

80
[ 0.28125   +0.25j        0.49109457+0.08831644j  0.2627594 -0.2867337j
  0.35049264-0.04397126j  0.44754537+0.1970449j  -0.10417022+0.28846002j
 -0.19616502+0.11586283j -0.18161989+0.32307393j  0.5517767 +0.38036165j
  0.05046208+0.12046704j -0.33607053-0.26599284j  0.27210546-0.07465169j
 -0.24003082+0.05373907j -0.21452243+0.10674819j -0.04600871+0.14358631j
  0.0626994 +0.08736775j  0.53125   +0.4375j     -0.39825862+0.66254641j
  0.43680491+0.3668746j   0.01595144+0.18609162j -0.2835711 -0.071391j
 -0.24169586+0.60786338j  0.02588168+0.21793871j  0.02111343-0.15019061j
  0.34911165+0.55713835j  0.32420587-0.25392738j -0.22744271+0.10062154j
  0.28612407+0.33458445j  0.02822113+0.071391j    0.28937808-0.72679748j
  0.15741834-0.5151882j   0.66245054-0.31682454j  0.03125   +0.125j
 -0.49864865-0.35884965j  0.01231268+0.12659221j -0.49654335-0.59522007j
  0.10109054-0.05373907j  0.60669567-0.17715569j  0.05676237+0.17994809j
  0.38409088+0.09987563j  0.1982233 +0.55713835j -0.1265

In [41]:
def channel(signal):
  convolved = np.convolve(signal, channelResponse)
  signal_power = np.mean(abs(convolved**2))
  sigma2 = signal_power * 10**(-SNRdb/10)  # calculate noise power based on signal power and SNR
  print ("RX Signal power: %.4f. Noise power: %.4f" % (signal_power, sigma2))
  # Generate complex noise with given variance
  noise = np.sqrt(sigma2/2) * (np.random.randn(*convolved.shape)+1j*np.random.randn(*convolved.shape))
  return convolved + noise

OFDM_TX = OFDM_withCP
OFDM_RX = channel(OFDM_TX)


RX Signal power: 0.2020. Noise power: 0.0008


In [39]:
print(OFDM_TX)

[ 0.28125   +0.25j        0.49109457+0.08831644j  0.2627594 -0.2867337j
  0.35049264-0.04397126j  0.44754537+0.1970449j  -0.10417022+0.28846002j
 -0.19616502+0.11586283j -0.18161989+0.32307393j  0.5517767 +0.38036165j
  0.05046208+0.12046704j -0.33607053-0.26599284j  0.27210546-0.07465169j
 -0.24003082+0.05373907j -0.21452243+0.10674819j -0.04600871+0.14358631j
  0.0626994 +0.08736775j  0.53125   +0.4375j     -0.39825862+0.66254641j
  0.43680491+0.3668746j   0.01595144+0.18609162j -0.2835711 -0.071391j
 -0.24169586+0.60786338j  0.02588168+0.21793871j  0.02111343-0.15019061j
  0.34911165+0.55713835j  0.32420587-0.25392738j -0.22744271+0.10062154j
  0.28612407+0.33458445j  0.02822113+0.071391j    0.28937808-0.72679748j
  0.15741834-0.5151882j   0.66245054-0.31682454j  0.03125   +0.125j
 -0.49864865-0.35884965j  0.01231268+0.12659221j -0.49654335-0.59522007j
  0.10109054-0.05373907j  0.60669567-0.17715569j  0.05676237+0.17994809j
  0.38409088+0.09987563j  0.1982233 +0.55713835j -0.1265612

In [40]:
print(OFDM_RX)

[ 0.29337583+0.247772j    0.48541976+0.08789225j  0.30817042-0.13824407j
  0.49411707+0.14046404j  0.61641648+0.18564271j  0.02999922+0.40535746j
 -0.11215583+0.31483028j -0.28369974+0.40347067j  0.49638984+0.33800001j
 -0.11587793+0.1713503j  -0.30402539+0.03799634j  0.25665004-0.02464728j
 -0.24582011-0.1285725j  -0.08027985+0.18644079j -0.11980993+0.09089522j
 -0.03029085+0.00705693j  0.48625827+0.45530565j -0.39143542+0.6682189j
  0.48609331+0.6443379j  -0.2784011 +0.29685171j -0.24088422+0.17172158j
 -0.29282738+0.65333898j -0.06416714+0.10932701j -0.23747784-0.04342991j
  0.27036574+0.65971512j  0.38638281-0.26751561j -0.30639071+0.37159839j
  0.47224626+0.40813088j -0.06818446+0.02592014j  0.29650279-0.54966818j
  0.15032458-0.47834386j  0.96132449-0.45504045j  0.22863565+0.06158686j
 -0.22786085-0.2420053j  -0.01619034+0.15118907j -0.53810463-0.82938669j
  0.07149282-0.02350034j  0.62473394-0.4993149j   0.07037748+0.17180948j
  0.65286929+0.20194288j  0.11818721+0.62592678j -0.